In [31]:
import base64
from itertools import cycle

def xor_encrypt(data: str, password: str) -> bytes:
    return bytes([b ^ ord(p) for b, p in zip(data.encode(), cycle(password))])

def xor_decrypt(data: bytes, password: str) -> str:
    return ''.join([chr(b ^ ord(p)) for b, p in zip(data, cycle(password))])

def encode_api_key(api_key: str, password: str) -> str:
    xor_bytes = xor_encrypt(api_key, password)
    return base64.urlsafe_b64encode(xor_bytes).decode()

def decode_api_key(encoded: str, password: str) -> str:
    xor_bytes = base64.urlsafe_b64decode(encoded)
    return xor_decrypt(xor_bytes, password)
# encode_api_key(apikey,encyptkey)



In [32]:
from dotenv import load_dotenv
import os
import getpass
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END
from langchain_community.tools.tavily_search import TavilySearchResults
from typing import TypedDict, List
load_dotenv()



True

In [6]:
key=getpass.getpass('Enter Encypt/Decrypt Key>?')
for i in filter(lambda x:x.endswith('API_KEY'),os.environ):
    print('Decrypting ..',i)
    os.environ[i]=decode_api_key(os.environ.get(i), key)

Enter Encypt/Decrypt Key>? ········


Decrypting .. GEMINI_API_KEY


## Decrypt Keys

In [4]:
import google.generativeai as genai
import os
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
llm = genai.GenerativeModel('gemini-1.5-flash-latest')

tool = TavilySearchResults(
    max_results=5,
    # include_answer=True,
    # include_raw_content=True,
    # include_images=True,
    # search_depth="advanced",
    # include_domains = []
    # exclude_domains = []
    api_key=os.environ.get('TAVILY_API_KEY')
)
# search_tool = TavilySearchResults(max_results=5, api_key=userdata.get('TAVILY_API_KEY'))


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_928/1301793142.py:6: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-tavily package and should be used instead. To use it run `pip install -U :class:`~langchain-tavily` and import as `from :class:`~langchain_tavily import TavilySearch``.
  tool = TavilySearchResults(


In [5]:
print(dir(llm))

['__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', '_async_client', '_client', '_generation_config', '_get_tools_lib', '_model_name', '_prepare_request', '_safety_settings', '_system_instruction', '_tool_config', '_tools', 'cached_content', 'count_tokens', 'count_tokens_async', 'from_cached_content', 'generate_content', 'generate_content_async', 'model_name', 'start_chat']


In [7]:
# response = llm.generate_content('Hi')

In [14]:
class State(TypedDict):
    destination: str
    dates: str
    outline: str
    research: List[str]
    itinerary: str


# 1. Instantiate graph with state schema
graph = StateGraph(State)


# 2. Define node functions
def plan_node(state: State) -> dict:
    prompt = (
        f"You are expert travel planner. Create a 5‑day outline for "
        f"{state['destination']} ({state['dates']})."
    )
    outline = llm.generate_content(prompt)
    # print(outline)
    outline=outline.text
    return {"outline": outline}

def research_node(state: State) -> dict:
    items = []
    print('state["outline"]:',state["outline"])
    for day in state["outline"].splitlines():
        print('@@@ ',day)
        # q = f"{state['destination']} {day} highlights"
        # print("DEBUG SEARCH RESULT:", tool.invoke(q))
        # snippet = tool.invoke(q)[0].get("snippet", "")
        # items.append(snippet)
        print(items)
    return {"research": items}

def generate_node(state: State) -> dict:
    content = "\n".join(state["research"])
    prompt = (
        f"Outline:\n{state['outline']}\n\nResearch:\n{content}\n\n"
        "Generate a detailed day‑by‑day itinerary."
    )
    print(prompt)
    return {"itinerary": llm.generate_content(prompt)}

# 3. Add nodes to graph
graph.add_node("plan", plan_node)
graph.add_node("gather_research", research_node)   # ✅ renamed to avoid conflict
graph.add_node("generate", generate_node)

# 4. Connect nodes with edges
graph.add_edge(START, "plan")
graph.add_edge("plan", "gather_research")          # ✅ updated edge
graph.add_edge("gather_research", "generate")
graph.add_edge("generate", END)

# 5. Compile graph
planner = graph.compile()
print(graph)
print(dir(planner))

['InputType', 'OutputType', '__abstractmethods__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__or__', '__orig_bases__', '__orig_class__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__ror__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_abatch_with_config', '_abc_impl', '_acall_with_config', '_aprepare_state_snapshot', '_atransform_stream_with_config', '_batch_with_config', '_call_with_config', '_defaults', '_migrate_checkpoint', '_prepare_state_snapshot', '_repr_mimebundle_', '_transform_stream_with_config', 'abatch', 'abatch_as_completed', 'abulk_update_state', 'aclear_cache', 'aget_graph', 'aget_state', 'aget_state_history', 'aget_subgraphs', 'ainvoke', 'as_tool', 'assign', 'astream',

In [15]:
state=State({
        "destination": input("Destination: "),
        "dates": input("Dates (e.g. July 1‑5): "),
        "outline": "",
        "research": [],
        "itinerary": ""
    })
final_state = planner.invoke(state)
print("\n🗓️ Outline:\n", final_state["outline"])
print("\n🔍 Research:\n", "\n".join(final_state["research"]))
print("\n📋 Itinerary:\n", final_state["itinerary"])

Destination:  Balli
Dates (e.g. July 1‑5):  June 1 -10


state["outline"]: ## Bali 5-Day Itinerary (June 1st-5th) -  A Blend of Culture, Nature & Relaxation

This itinerary balances cultural experiences, stunning natural beauty, and relaxation, suitable for a moderate activity level. It can be adjusted to your preferences – more adventure, less relaxation, etc.  Crucially, **June falls within the dry season**, offering optimal weather conditions.

**Day 1 (June 1st): Arrival in Seminyak & Beach Bliss**

* **Morning (8:00 AM):** Arrive at Denpasar Airport (DPS), transfer to your pre-booked accommodation in Seminyak.  Consider a private transfer for convenience, especially if arriving with luggage.
* **Late Morning (10:00 AM):** Settle in, then head to Seminyak Beach. Relax, soak up the sun, maybe take a dip in the ocean.  Seminyak offers a range of beach clubs for lunch with varying levels of luxury.
* **Afternoon (2:00 PM):** Lunch at a beachside Warung (local eatery) for authentic Indonesian cuisine.  Explore the shops and boutiques along S

In [22]:
from langgraph.graph import StateGraph, START, END


In [23]:
class state(TypedDict):
    document:str
    question:str
    output:str



In [26]:
def documentFunction(state):
    print('Input State under documentFunction:',state)
    return {'document':'This is my Document'}

def documentQuestion(state):
    print('Input State under documentQuestion',state)
    return {'question':'TestQuestion'}

graph=StateGraph(state)
graph.add_node('documentNode',documentFunction)
graph.add_node('QuestionNode',documentQuestion)

graph.add_edge(START,'documentNode')
graph.add_edge('documentNode','QuestionNode')
graph.add_edge('QuestionNode',END)

planner=graph.compile()

In [28]:
planner.invoke(State())

Input State under documentFunction: {}
Input State under documentQuestion {'document': 'This is my Document'}


{'document': 'This is my Document', 'question': 'TestQuestion'}